In [9]:
from index import build_prompt, handler, handle_inputs, extract_variables
import copy
import json

example_inputs = {"gameDescription":"""In this action-packed adventure, players take on the role of a ninja warrior on a quest to save the kingdom from an evil sorcerer. With fast-paced gameplay and challenging obstacles, players must use their skills and special abilities to navigate through different environments and defeat the sorcerer's minions. The ultimate goal is to reach the sorcerer's castle and engage in an epic battle to save the kingdom from darkness. Along the way, players will encounter friendly characters who will aid them in their journey, as well as powerful enemies who will stop at nothing to protect the sorcerer. Will you have what it takes to defeat the sorcerer and restore peace to the kingdom? It's time to find out.""","gameType":"Action-adventure","graphicView":"Side scroller","graphicStyle":"Fantasy, hand drawn","multiplayerOption":"Single player, local multiplayer","plot":"The kingdom has been plunged into darkness by an evil sorcerer who seeks to gain ultimate power. As the last remaining ninja warrior, it is up to the player to defeat the sorcerer and save the kingdom. Along the way, they will encounter various challenges and obstacles, from treacherous terrain to powerful enemies. With the help of their ninja skills and special abilities, the player must navigate through different environments and battle their way to the sorcerer's castle. But the sorcerer is not alone, as he has summoned powerful creatures to defend him. The player must use all their skills and wit to defeat the sorcerer and restore peace to the kingdom.","characters":[{"name":"ninja warrior","player":"player","description":"agile and skilled"},{"name":"sorcerer","player":"enemy","description":"powerful and menacing"},{"name":"magical creatures","player":"enemy","description":"various types and abilities"},{"name":"king","player":"friendly","description":"wise and kind"},{"name":"queen","player":"friendly","description":"brave and resourceful"}],"colourSchemes":[{"colour":"black","label":"main colour"},{"colour":"red","label":"accent colour"},{"colour":"green","label":"accent colour"},{"colour":"purple","label":"accent colour"},{"colour":"gold","label":"accent colour"}],"styleKeywords":"Fantasy, action-packed, magical, adventurous, fast-paced, challenging, epic, mystical, heroic, mythical","environments":[{"name":"forest","description":"a mystical forest filled with magical creatures"},{"name":"mountains","description":"treacherous mountains with steep cliffs and icy terrain"},{"name":"cave","description":"a dark and dangerous cave with hidden treasures"},{"name":"village","description":"a peaceful village with friendly inhabitants"},{"name":"sorcerer's castle","description":"a towering castle with powerful defenses)"}]}

In [10]:
example_inputs_without_one = copy.copy(example_inputs)
example_inputs_without_one.pop('characters')
example_inputs_without_one.pop('environments')
example_inputs_without_one.pop('gameDescription')
event = {
    'body': json.dumps(example_inputs_without_one),
}

body, missing_options, present_options = handle_inputs(event)

items_prompt, prompt = build_prompt(body, missing_options, present_options)



In [3]:
print(prompt)

You are a script writer for a 2D video game company. You have been given a game description, the graphic view of the game (for instance isometric/top down/side scroller), the game type (for instance action/adventure/role playing) and the multiplayer option (for instance single player/online multiplayer/local multiplayer).

game description (minimum 100 words).

In this action-packed adventure, players take on the role of a ninja warrior on a quest to save the kingdom from an evil sorcerer. With fast-paced gameplay and challenging obstacles, players must use their skills and special abilities to navigate through different environments and defeat the sorcerer's minions. The ultimate goal is to reach the sorcerer's castle and engage in an epic battle to save the kingdom from darkness. Along the way, players will encounter friendly characters who will aid them in their journey, as well as powerful enemies who will stop at nothing to protect the sorcerer. Will you have what it takes to defe

In [4]:
def extract_variables(response, missing_options, item_prompts):
    # Trim the final *** from the response
    variables = {}
    for option in missing_options:
        variables[option] = ''
    response_lines = response.split('\n')
    current_line_idx = 0
    option_starting_lines = [item_prompts[option] for option in missing_options]

    for i, option_starting_line in enumerate(option_starting_lines):
        print(f'option starting line: {option_starting_line}')
        while True:
            this_line = response_lines[current_line_idx]
            if not this_line.strip():
                pass
            elif response_lines[current_line_idx][0].strip().isdigit():
                break
            print(f'{response_lines[current_line_idx]}')
            print(f'Not found starting line. Advancing')
            current_line_idx += 1
        print('Found starting line at idx: ' + str(current_line_idx))
        print(f'Current line: {response_lines[current_line_idx]}')

        variable_lines = []
        while True:
            current_line_idx += 1
            if '***' in response_lines[current_line_idx]:
                print(f'Adding line: {response_lines[current_line_idx]}')
                variable_lines.append(response_lines[current_line_idx].replace('***', ''))
                break
            else:
                if not response_lines[current_line_idx]:
                    continue
                elif response_lines[current_line_idx][0].strip().isdigit():
                    break
                else:
                    print(f'Adding line: {response_lines[current_line_idx]}')
                    variable_lines.append(response_lines[current_line_idx])

        print(f'Stopping at line: {response_lines[current_line_idx]}')
        print(f'Variable {missing_options[i]}: {variable_lines}')
        variables[missing_options[i]] = ' '.join(variable_lines)
    return variables


In [25]:
# output = handler(event, None)
response = "1. game description:\n\nIn this top-down, pixel art adventure game, you take on the role of a brave knight on a mission to rescue the captive Princess from an evil wizard, Zorag. You have to cross through a lush green forest teeming with wildlife, traverse treacherous mountain terrain with avalanches, and finally enter Zorag's dark and foreboding castle tower. Along your journey, face-off against Zorag’s minions and challenge yourself to solve cryptic puzzles. The game's style is medieval and fantasy, capturing the spirit of classic platformer and adventure games. The colour scheme of the game is predominantly emerald green complemented with accents of gold, enhancing the lush, rich environments and fitting perfectly to the valiant persona of our knight. You play alone in this single player game, where adventure, action and courage awaits you.\n\n2. game type (e.g. adventure, puzzle, shoot'em up):\n\nAdventure, Action, Platformer.***"
missing_variables = extract_variables(response, missing_options, items_prompt)

option starting line: game description:
Found starting line at idx: 0
Current line: 1. game description:
Adding line: In this top-down, pixel art adventure game, you take on the role of a brave knight on a mission to rescue the captive Princess from an evil wizard, Zorag. You have to cross through a lush green forest teeming with wildlife, traverse treacherous mountain terrain with avalanches, and finally enter Zorag's dark and foreboding castle tower. Along your journey, face-off against Zorag’s minions and challenge yourself to solve cryptic puzzles. The game's style is medieval and fantasy, capturing the spirit of classic platformer and adventure games. The colour scheme of the game is predominantly emerald green complemented with accents of gold, enhancing the lush, rich environments and fitting perfectly to the valiant persona of our knight. You play alone in this single player game, where adventure, action and courage awaits you.
Stopping at line: 2. game type (e.g. adventure, pu

In [26]:
missing_variables

{'gameDescription': "In this top-down, pixel art adventure game, you take on the role of a brave knight on a mission to rescue the captive Princess from an evil wizard, Zorag. You have to cross through a lush green forest teeming with wildlife, traverse treacherous mountain terrain with avalanches, and finally enter Zorag's dark and foreboding castle tower. Along your journey, face-off against Zorag’s minions and challenge yourself to solve cryptic puzzles. The game's style is medieval and fantasy, capturing the spirit of classic platformer and adventure games. The colour scheme of the game is predominantly emerald green complemented with accents of gold, enhancing the lush, rich environments and fitting perfectly to the valiant persona of our knight. You play alone in this single player game, where adventure, action and courage awaits you.",
 'gameType': 'Adventure, Action, Platformer.'}

In [11]:
output = handler(event, None)

Body: {'gameType': 'Action-adventure', 'graphicView': 'Side scroller', 'graphicStyle': 'Fantasy, hand drawn', 'multiplayerOption': 'Single player, local multiplayer', 'plot': "The kingdom has been plunged into darkness by an evil sorcerer who seeks to gain ultimate power. As the last remaining ninja warrior, it is up to the player to defeat the sorcerer and save the kingdom. Along the way, they will encounter various challenges and obstacles, from treacherous terrain to powerful enemies. With the help of their ninja skills and special abilities, the player must navigate through different environments and battle their way to the sorcerer's castle. But the sorcerer is not alone, as he has summoned powerful creatures to defend him. The player must use all their skills and wit to defeat the sorcerer and restore peace to the kingdom.", 'colourSchemes': '[{"colour": "black", "label": "main colour"}, {"colour": "red", "label": "accent colour"}, {"colour": "green", "label": "accent colour"}, {

In [4]:
json.loads(output['body'])

{'characters': [{'name': 'Ninja Warrior',
   'player': True,
   'description': 'The main character of the game, a skilled ninja warrior on a quest to save the kingdom.'},
  {'name': 'Sorcerer',
   'player': False,
   'description': 'The main antagonist of the game, an evil sorcerer seeking ultimate power.'},
  {'name': 'Friendly Characters',
   'player': False,
   'description': 'Various characters throughout the game who will aid the player on their journey.'},
  {'name': 'Minions',
   'player': False,
   'description': "The sorcerer's loyal followers who will stop at nothing to protect their master."}],
 'colourSchemes': [{'colour': 'Midnight Blue',
   'label': "Represents the darkness and danger of the sorcerer's kingdom."},
  {'colour': 'Crimson',
   'label': 'Symbolizes the bloodshed and battles throughout the game.'},
  {'colour': 'Emerald',
   'label': 'Represents the magic and mystical elements of the game.'},
  {'colour': 'Goldenrod',
   'label': "Symbolizes the player's brave